This notebook's goal is to retrieve data from ADFG Alaska, in this case the CSIS - Community Subsistence Information System - database. There is no easy tool to retrieve all informations at a Community level so a method was found to do it through HTML request, this shouldn't be run in parallel as it could shut down the ADFG servers.

Once the data are retrieve, the goal is to categorize a community's subsistence profile based on the subsistence statistics published on the CSIS database.


In [1]:
import os
import pandas as pd
from multiprocessing.dummy import Pool as ThreadPool #It would greatly reduce the time with post processing but the ADFG server can't handle multiple request

In [2]:
def dit( url ):
    try :
        df = pd.read_excel(url,engine='xlrd')
        _ = df.to_csv('com-{}.csv'.format(url[85:-26]))
        return df
    except : print('FAILED {}'.format(url[85:-26]))

A range of 0 to 1000 is used as the community ID are not linear

The HTML request was extracted/found with Firebug on Firefox using event recording, Beautiful Soup could have been used but the structure of the data is really messy

In [3]:
url = ['http://www.adfg.alaska.gov/sb/CSIS/index.cfm?ADFG=excelDownloads.HarvestExcel&CommID={}&allYears&Category=Harvest'.format(i) for i in range(0,1000)]

In [4]:
# pool = ThreadPool(2)
# results = pool.map(dit,url[0:1000])
# pool.close()
# pool.join()
# df = pd.concat(results)

#Using the saved dataset to not have to extract again
df = pd.read_csv('~/DL/master_CSIS.csv')

In [5]:
df.columns

Index(['Unnamed: 0', 'Project_ID', 'Project_Name', 'Community_Code',
       'Community_Name', 'Study_Year', 'Resource_Code', 'Resource',
       'Species_Hierarchy', 'Percent_Attempting_to_Harvest',
       'Percent_Harvesting', 'Percent_Using', 'Percent_Giving_Away',
       'Percent_Receiving', 'Units', 'Reported_Harvest', 'Conversion_Factor',
       'Reported_Pounds_Harvested', 'Average_Lbs_Harvested_per_Household',
       'Estimated_Harvest', 'Lower_Harvest_Amount_Estimate',
       'Upper_Harvest_Amount_Estimate', 'Estimated_Pounds_Harvested',
       'Lower_Harvest_Lbs_Estimate', 'Upper_Harvest_Lbs_Estimate',
       'Per_Capita_Lbs_Harvested', 'Percent_95_CIP',
       'Percent_Of_Total_Harvest', 'Mean_Per_Capita_Lbs_Used',
       'Mean_Per_Capita_Grams_per_Day',
       'NinetyFifth_Confidence_Limit_of_Mean_Use_in_Pounds',
       'NinetyFifth_Confidence_Limit_of_Mean_Use_in_Grams_per_Day',
       'NinetyFifth_Confidence_Limit_of_Mean_Use_Grams_per_Day',
       'Fiftieth_Percentile_Use_

In [6]:
df.shape

(148444, 37)

In [7]:
df['Code'] = pd.to_numeric([str(i)[0] for i in df['Resource_Code']])

In [8]:
df1 = df[['Community_Name','Code','Estimated_Pounds_Harvested']].copy()
df1 = df1[(df1['Code']!=0) & (df1['Code']!=9)]


In [101]:
r = df1.groupby(by=['Community_Name','Code']).sum()
r.head()

Estimated_Pounds_Harvested
Community_Name Code                            
Adak           3                         6864.0
Akhiok         1                       381438.0
               2                        42633.0
               3                        74668.0
               4                         7648.0

In [102]:
r = r.unstack(level=1)
r.head()

Estimated_Pounds_Harvested                             \
Code                                    1            2             3   
Community_Name                                                         
Adak                                  NaN          NaN   6864.000000   
Akhiok                       3.814380e+05   42633.0000  74668.000000   
Akiachak                     2.912090e+06  431622.0000  43363.000000   
Akiak                        8.037090e+05  101900.0000   5007.000000   
Akutan                       2.302708e+05   12179.9998  82038.555424   

                                                           
Code                        4             5             6  
Community_Name                                             
Adak                      NaN           NaN           NaN  
Akhiok            7648.000000  54025.000000   1735.000000  
Akiachak        281428.000000      0.000000  63614.000000  
Akiak            41049.000000    192.000000  24550.000000  
Akutan           24291.107093  21486.455633   6104.583372

Now that we have what we want, let's just clean up and give appropriate title and calculate percentages

In [11]:
r.columns = [df['Resource'][df['Resource_Code']==code].values[0] for code in [i * pow(10,8) for i in range(1,7)]]
for i in r.columns:
    r['{}-perc'.format(i)] = (r[i]/r.sum(axis=1))*100


Then let's drop all lines filled with Nan and reset to 0 all the lines with just a few NaN

In [17]:
finale = r[[i for i in r.columns[6:12]]].dropna(axis=0,how='all').fillna(0)
finale.index

Index(['Adak', 'Akhiok', 'Akiachak', 'Akiak', 'Akutan', 'Alakanuk', 'Alatna',
       'Aleknagik', 'Allakaket', 'Allakaket/Alatna',
       ...
       'Wainwright', 'Wales', 'West Glenn Highway', 'Whale Pass',
       'White Mountain', 'Whitestone Logging Camp', 'Whittier', 'Wiseman',
       'Wrangell', 'Yakutat'],
      dtype='object', name='Community_Name', length=277)

In order to build spatial vizualisation we need to find the geographic coordinates for all the communities, we use geopy to do so as it is pretty effective even for Alaska.

In [14]:
def coordinate(community) :
    from geopy.geocoders import Nominatim
    geolocator = Nominatim()
    comm = '{} Alaska'.format(community)
    try :
        return (community , geolocator.geocode(comm).latitude , geolocator.geocode(comm).longitude)
    except :
        print('Failed {} '.format(community))

Using multiprocessing to speed up the process and raising a flag for failed queries, most of the missing communities are of low importance so we will just ignore them

In [46]:
pool = ThreadPool(32)
results = pool.map(coordinate,finale.index)
pool.close()
pool.join()


Failed Fritz Creek Census Designated Place 
Failed Game Creek Census Designated Place 
Failed Hurricane-Broad Pass 
Failed Paxson-Sourdough Failed Nunam Iqua (Sheldon Point) 

Failed Mentasta Pass 
Failed Pilot Point / Ugashik 
Failed Slana Homestead North 
Failed Slana Homestead South 
Failed Port San Juan (Evans Island) 
Failed South Wrangell Mountains 


This step seem to be necessary to get the tiles properly working on bokeh, just converting the WSG84 coordinates into web mercator

In [79]:
from pyproj import Proj, transform
results = [result for result in results if result!= None]
labels = ['place', 'Y', 'X']
geo = pd.DataFrame.from_records(results, columns=labels,index='place')
inProj = Proj(init='epsg:4326')
outProj = Proj(init='epsg:3857')

geo['X'],geo['Y'] = transform(inProj,outProj,geo['X'].values,geo['Y'].values )


In [80]:
full = pd.concat([finale,geo],axis=1)

In [81]:
from bokeh.plotting import figure, output_notebook, show
output_notebook()
p = figure(title="My first interactive plot!")
x_coords = full['X']
y_coords = full['Y']
p.circle(x=x_coords, y=y_coords, size=10, color="red")

Loading BokehJS ...

GlyphRenderer(id='4a0ca1a1-a85f-42cf-84e8-248d8aedda55', ...)

In [82]:
show(p)

In [94]:
del p

In [98]:
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.plotting import figure, output_notebook, show, save
from bokeh.tile_providers import STAMEN_TERRAIN
my_hover = HoverTool()
output_notebook()
source = ColumnDataSource(full)
my_hover = HoverTool()
my_hover.tooltips = [
    ("Place", "@index"),
    ("Fish", '@{Fish-perc}'),
    ("Land Mammal", '@{Land Mammals-perc}'),
    ("Bird and Eggs", '@{Birds and Eggs-perc}'),
    ("Marine Mammal", "@{Marine Mammals-perc}"),
    ("Marine Invertebrates", '@{Marine Invertebrates-perc}'),
    ("Vegetation", "@{Vegetation-perc}")
]
p = figure(title="CSIS database visualisation")
x_coords = full['X']
y_coords = full['Y']
p.circle(x='X', y='Y', source = source , size=5, color="red")
p.add_tools(my_hover)
p.add_tile(STAMEN_TERRAIN)

Loading BokehJS ...

TileRenderer(id='363dcdf5-ec36-49a1-80fb-1385d20bf4f7', ...)

In [99]:
show(p)

This tutorial was used as inspiration for this piece of code https://automating-gis-processes.github.io/2016/Lesson5-interactive-map-bokeh.html